In [69]:
import pandas as pd
import numpy as np
emails_subject = pd.read_csv('./emails/dbworld_subjects_stemmed.csv').iloc[:,1:]
emails_body = pd.read_csv('./emails/dbworld_bodies_stemmed.csv').iloc[:,1:]

In [23]:
emails_subject.head()

,10th,13th,1st,2nd,31st,3rd,5th,6th,abstract,academ,...,vacanc,valencia,vehicular,video,web,wireless,workflow,workshop,zurich,CLASS
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
from sklearn.model_selection import train_test_split
Xs_train,Xs_test, ys_train, ys_test = train_test_split(emails_subject.iloc[:,:-1],emails_subject['CLASS'],test_size=0.2, random_state=0)
Xb_train,Xb_test, yb_train, yb_test = train_test_split(emails_body.iloc[:,:-1],emails_body['CLASS'],test_size=0.2, random_state=0)

# Self - Implement with laplas smoothing k = 1

In [118]:
import numpy as np
def Naive_Bayes_Classifier(X_train, y_train,k=1):
    words_dict = X_train.columns.values
    Class = y_train.name
    prob_0 = {}
    prob_1 = {}
    train_data = X_train.join(y_train)
    for word in words_dict:
        countX0, countX1 = train_data.loc[:,[word,Class]].groupby(Class).sum().values
        prob_0[word] = (countX0[0]+k)/(train_data.shape[0]+np.unique(ys_train).shape[0]*k)
        prob_1[word] = (countX1[0]+k)/(train_data.shape[0]+np.unique(ys_train).shape[0]*k)
        prob_n = k/(train_data.shape[0]+np.unique(ys_train).shape[0]*k)
    return prob_0,prob_1,prob_n
    
def predict(X_train, y_train, X_test):
    result = []
    words_dict = X_train.columns.values
    prob_0,prob_1,prob_n = Naive_Bayes_Classifier(X_train, y_train,k=1)
    for i in range(X_test.shape[0]):
        words = X_test.iloc[i][X_test.iloc[i] != 0].index
        pred_0 = 1
        pred_1 = 1
        for word in words:
            if word in words_dict:
                pred_0 = prob_0[word]*pred_0
                pred_1 = prob_1[word]*pred_1
            if word not in words_dict:
                pred_0 = prob_n*pred_0
                pred_1 = prob_n*pred_1
        if pred_0 >= pred_1:
            result.append(0)
        else:
            result.append(1)
    return result

In [125]:
ys_pred = predict(Xs_train, ys_train, Xs_test)

In [126]:
from sklearn import metrics
print("subject Precision", metrics.precision_score(ys_test, ys_pred))
print( "subject Recall", metrics.recall_score(ys_test, ys_pred))
print( "subject f1_score", metrics.f1_score(ys_test, ys_pred))

subject Precision 0.8
subject Recall 1.0
subject f1_score 0.888888888888889


In [127]:
yb_pred = predict(Xb_train, yb_train, Xb_test)

In [128]:
from sklearn import metrics
print("body Precision", metrics.precision_score(yb_test, yb_pred))
print( "body Recall", metrics.recall_score(yb_test, yb_pred))
print( "body f1_score", metrics.f1_score(yb_test, yb_pred))

body Precision 0.8571428571428571
body Recall 0.75
body f1_score 0.7999999999999999


# scikit-learn implementation

In [129]:
from sklearn.naive_bayes import MultinomialNB

In [130]:
clf = MultinomialNB()
clf.fit(Xs_train, ys_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [132]:
ys_pred = clf.predict(Xs_test)

In [133]:
from sklearn import metrics
print("subject Precision", metrics.precision_score(ys_test, ys_pred))
print( "subject Recall", metrics.recall_score(ys_test, ys_pred))
print( "subject f1_score", metrics.f1_score(ys_test, ys_pred))

subject Precision 0.8
subject Recall 1.0
subject f1_score 0.888888888888889


In [134]:
clf = MultinomialNB()
clf.fit(Xb_train, yb_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [135]:
yb_pred = clf.predict(Xb_test)

In [136]:
from sklearn import metrics
print("body Precision", metrics.precision_score(yb_test, yb_pred))
print( "body Recall", metrics.recall_score(yb_test, yb_pred))
print( "body f1_score", metrics.f1_score(yb_test, yb_pred))

body Precision 0.8
body Recall 1.0
body f1_score 0.888888888888889
